# Application of Convolutional Neural Network (CNN)

CNN is a type of neural network model that is particularly suited for image recognition and processing tasks, it is used to extract higher representations for the image content.
### Goal: Classify images using downloaded FashionMNIST data.

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import transforms

In [2]:
# Model architecture definition. Use the torch built-in functions.

dim_inputs, num_outputs, = 784, 10

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # 1. Complete layer definition for a 2-layer CNN
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels = 1,              
                out_channels = 16,            
                kernel_size = 5,              
                stride = 1,                   
                padding = 2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size = 2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 16, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.out = nn.Linear(dim_inputs, num_outputs)   

    def forward(self, x):
        
        # 2. Complete network architecture definition
        x = self.conv1(x)
        x = self.conv2(x)
        
        # Flatten the output of conv2
        x = x.view(x.size(0), -1)       
        logits = self.out(x)
        
        return logits

In [3]:
# Data Preparation 

trans = transforms.ToTensor()
fmnist_train = torchvision.datasets.FashionMNIST(
    root ="../data", train = True, transform = trans, download = True)
fmnist_test = torchvision.datasets.FashionMNIST(
    root ="../data", train = False, transform = trans, download = True)
BATCH_SIZE = 32
trainloader = torch.utils.data.DataLoader(fmnist_train, batch_size = BATCH_SIZE,
                                          shuffle = True, num_workers = 2)   
testloader = torch.utils.data.DataLoader(fmnist_test, batch_size=BATCH_SIZE,
                                         shuffle = False, num_workers = 2)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



In [4]:
learning_rate = 0.001
num_epochs = 10

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = CNN()   # create a model
model = model.to(device)

cpu


In [5]:
# Compute Accuracy

def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [6]:
criterion = nn.CrossEntropyLoss()  # define loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  # optimizer

# The training process
for epoch in range(num_epochs):
    train_running_loss = 0.0
    train_acc = 0.0

    model = model.train()   # training mode

    ## training step
    for i, (images, labels) in enumerate(trainloader):
        
        images = images.to(device)
        labels = labels.to(device)

        ## forward + backprop + loss
        logits = model(images)  # forward propagation
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()

        ## update model params
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += get_accuracy(logits, labels, BATCH_SIZE)
    
    model.eval()  # evaluation mode
    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
          %(epoch, train_running_loss / i, train_acc/i))  # current training accuracy
    
    ## evaluation during training to monitor it
    test_acc = 0.0
    for i, (images, labels) in enumerate(testloader, 0):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_acc += get_accuracy(outputs, labels, BATCH_SIZE)
        
    print('Test Accuracy: %.2f'%( test_acc/i))

Epoch: 0 | Loss: 0.4922 | Train Accuracy: 82.35
Test Accuracy: 87.00
Epoch: 1 | Loss: 0.3307 | Train Accuracy: 88.31
Test Accuracy: 87.69
Epoch: 2 | Loss: 0.2911 | Train Accuracy: 89.65
Test Accuracy: 89.75
Epoch: 3 | Loss: 0.2645 | Train Accuracy: 90.54
Test Accuracy: 88.31
Epoch: 4 | Loss: 0.2469 | Train Accuracy: 91.05
Test Accuracy: 90.60
Epoch: 5 | Loss: 0.2342 | Train Accuracy: 91.47
Test Accuracy: 90.71
Epoch: 6 | Loss: 0.2231 | Train Accuracy: 91.92
Test Accuracy: 90.08
Epoch: 7 | Loss: 0.2139 | Train Accuracy: 92.21
Test Accuracy: 90.42
Epoch: 8 | Loss: 0.2066 | Train Accuracy: 92.56
Test Accuracy: 90.88
Epoch: 9 | Loss: 0.1992 | Train Accuracy: 92.62
Test Accuracy: 91.13
